## CPSC 340 Lecture 21: kernel methods



## Demo of the "other" normal equations and the kernel trick

Consider the randomly generated dataset below:

In [192]:
import numpy as np
n = 20
d = 3
p = 2
λ = 1
np.random.seed(1)
X = np.random.randint(0,10,size=(n,d))
X

array([[5, 8, 9],
       [5, 0, 0],
       [1, 7, 6],
       [9, 2, 4],
       [5, 2, 4],
       [2, 4, 7],
       [7, 9, 1],
       [7, 0, 6],
       [9, 9, 7],
       [6, 9, 1],
       [0, 1, 8],
       [8, 3, 9],
       [8, 7, 3],
       [6, 5, 1],
       [9, 3, 4],
       [8, 1, 4],
       [0, 3, 9],
       [2, 0, 4],
       [9, 2, 7],
       [7, 9, 8]])

In [193]:
y = np.random.randint(-5, 5, n)
y

array([ 1,  4, -2,  2,  2, -1,  0,  4, -2,  1,  3, -5, -3,  2,  2,  4,  2,
       -2, -5,  3])

We decide to use a degree 2 polynomial basis to fit this dataset, using L2 regularization with $\lambda=1$. 

We then want to make a prediction on the test vector:

In [194]:
Xtest = np.array([[-1, 0, 1]])

Code this up 3 ways:

1. Using the "regular" normal equations
2. Using the "other" normal equations without the kernel trick
3. Using the "other" normal equations with the kernel trick

and show that you get the same answers all 3 ways.

In [195]:
from sklearn.preprocessing import PolynomialFeatures

#### Approach 1

In [200]:
def quad_features_3d(X):
    poly = PolynomialFeatures(degree=2)
    Z = poly.fit_transform(X)
    Z[:,[1,2,3,5,6,8]] *= np.sqrt(2)
    return Z

In [201]:
Z = quad_features_3d(X)

In [202]:
Z.shape

(20, 10)

Running time of the above step: 

- We have $O(d^p)$ features we're making. Let's call this $O(k)$ for now to keep it general and we can plug $k=d^p$ back in at the end.
- We need to transform each of the $n$ training examples. 
- For each training example, for each new feature, we potentially need to look at all the old features.
- $O(ndk)$

In [203]:
w = np.linalg.solve(Z.T@Z + λ*np.eye(poly.n_output_features_), Z.T@y)
w

array([-1.49169314,  1.21932561, -1.33559322,  0.10734793, -0.0292787 ,
       -0.02524845, -0.16662725,  0.14035137,  0.08089053,  0.04287204])

Running time of the above step:

- Multiply $Z^TZ$, meaning an $k \times n$ multiplied by $n \times k$ which is $O(nk^2)$
- Then the solving is $O(k^3)$ so we have $O(nk^2 + k^3)$ which we've seen in an earlier lecture.

In [209]:
Ztest = quad_features_3d(Xtest)
Ztest@w

array([-2.8150272])

Running time of the above step:

- $O(k)$
- So the total is $O(ndk+nk^2+k^3)$.

#### Approach 2

In [210]:
w_other = Z.T @ np.linalg.solve(Z@Z.T + λ*np.eye(n), y)
w_other

array([-1.49169314,  1.21932561, -1.33559322,  0.10734793, -0.0292787 ,
       -0.02524845, -0.16662725,  0.14035137,  0.08089053,  0.04287204])

Running time of the above step:

- Multiply $ZZ^T$, meaning an $n \times k$ multiplied by $k \times n$ which is $O(n^2k)$
- Then the solving is $O(n^3)$ so we have $O(n^2k + n^3)$ which we've seen in the lecture.

We can see we get the same $w$ so the prediction will also be the same.

In [211]:
Ztest = quad_features_3d(Xtest)
Ztest@w_other

array([-2.8150272])

#### Approach 3

$K(a, b) = (1+a^Tb)^p$

In [212]:
K = (1 + np.sum(X[None] * X[:,None], axis=-1))**p

In [213]:
K.shape

(20, 20)

- $K$ is $n \times n$ so it costs $O(n^2d)$

In [214]:
u = np.linalg.solve(K + λ*np.eye(n), y)

In [215]:
u.shape

(20,)

- Costs $n^3$ to solve the $n \times n$ system.

In [216]:
Ktest = (1 + np.sum(X[None] * Xtest[:,None], axis=-1))**p

In [217]:
Ktest.shape

(1, 20)

In [218]:
Ktest @ u

array([-2.8150272])

And we get the same prediction!

- Total cost $O(n^2d + n^3)$ which is independent of $k$.

Summary of running times:

- Regular approach $O(ndk+nk^2+k^3)$ 
- Other normal equations $O(ndk + n^2k + n^3)$
- Kernel trick $O(n^2d + n^3)$
  - Assuming kernel function costs $O(d)$ per pair

Also reproduce with sklearn kernel regression:

In [219]:
from sklearn.metrics import pairwise_kernels

In [228]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge(kernel="polynomial", degree=2, gamma=1)
kr.fit(X, y);

In [229]:
kr.predict(Xtest)

array([-2.8150272])

## Question

What is the **space complexity** of a trained model for each of the above 3 approaches?

**Mike's answer:**

- Approach 1: $O(k)$, just storing $w$
- Approach 2: $O(k)$, same as above
- Approach 3: $O(nd)$, need to store the training data

## Question

What is the **prediction time complexity** (for one example) for each of the above 3 approaches?

**Mike's answer:**

- Approach 1: $O(kd)$, the time to form `Ztest`. There's also the dot product which takes $O(k)$ but that's negligible.
- Approach 2: $O(kd)$, same as above
- Approach 3: $O(nd)$, need to run through the training data